In [13]:
import copy

# Read File
with open('astar_in.txt') as f:
    file = f.readlines()

# Separate the start state and goal state into two lists
start = []
goal = []
lines = len(file)//2
for line in range(1, lines):
    start.append(file[line].strip().split())
    goal.append(file[lines+line].strip().split())

# Based on a [size x size] puzzle
size = len(start)

In [14]:
class Node():
    def __init__(self, state, parent, operator):
        self.state = state
        self.parent = parent
        self.operator = operator
        self.cost = None

In [15]:
def SequenceRef(state):

    # Rotating elements before the edge
    clockwise_list = []
    clockwise_list.extend(state[0][:size-1])
    clockwise_list.extend([column[-1] for column in state[:size-1]])
    clockwise_list.extend(list(reversed(state[-1][1:size])))
    clockwise_list.extend(list(reversed([column[0] for column in state[1:size]])))

    # Pairing in nested list
    sequence = []
    empty_middle = True # Assume first that the empty tile is in the middle
    for index in range(len(clockwise_list)):
        if clockwise_list[index] == '*':  # Exclude the pair with empty first tile
            empty_middle = False # Empty tile is in the clockwise list
            continue
        sequence.append([clockwise_list[index], clockwise_list[(index+1)%8]])
    return sequence, empty_middle

In [16]:
def ComputeCost(node, S_goal, heuristic):

    # Path cost
    g_node = 0
    node_copy = copy.deepcopy(node)
    while node_copy.parent is not None:
        g_node += 1
        node_copy = node_copy.parent
    
    # Wrong tile and Manhattan distance calculation
    P_node = 0
    wrong_tile = 0
    for r_node in range(size):
        for c_node in range(size):
            tile = node.state[r_node][c_node]
            if tile != '*':
                for r_goal in range(size):
                    if tile in goal[r_goal]:
                        distance = abs(r_node - r_goal) + abs(c_node - goal[r_goal].index(tile))
                        if distance != 0:
                            wrong_tile += 1
                        P_node += distance
    
    # Number of tile in wrong position
    if heuristic == 'a':
        h_node = wrong_tile
        f_node = g_node + h_node
        return f_node, g_node, h_node
    
    # Manhattan distance
    if heuristic == 'b':
        h_node = P_node
        f_node = g_node + h_node
        return f_node, g_node, h_node
    
    # Nilsson's sequence score
    if heuristic == 'c':
        
        # Sequence score
        S_node = 0
        sequence, empty_middle = SequenceRef(node.state)
        if not empty_middle:
            S_node += 1
        for element in sequence:
            if element not in S_goal:
                S_node += 2

        # Nilsson's sequence score
        h_node = P_node + 3*S_node

        # Total path cost
        f_node = g_node + h_node
        
        return f_node, g_node, h_node, P_node, S_node

In [17]:
def SwapTile(tile1, tile2, state, operator):
    temp_state = copy.deepcopy(state)
    temp_state[tile1[0]][tile1[1]], temp_state[tile2[0]][tile2[1]] = state[tile2[0]][tile2[1]], state[tile1[0]][tile1[1]]
    return (operator, temp_state)

In [18]:
def NodeExpansion(state):

    # Find index of empty tile
    for s in state:
        if '*' in s:
            star = state.index(s), s.index('*')
            break

    # Possible moves of empty tile
    expand = []
    if star[0] != 0:
        tile = star[0]-1, star[1]
        expand.append(SwapTile(star, tile, state, 'up'))
    if star[0] != len(state)-1:
        tile = star[0]+1, star[1]
        expand.append(SwapTile(star, tile, state, 'down'))
    if star[1] != 0:
        tile = star[0], star[1]-1
        expand.append(SwapTile(star, tile, state, 'left'))
    if star[1] != len(state)-1:
        tile = star[0], star[1]+1
        expand.append(SwapTile(star, tile, state, 'right'))
    return expand

In [19]:
def NodeStatesCosts(nodes):
    states = []
    costs = []
    for node in nodes:
        states.append(node.state)
        costs.append(node.cost)
    return states, costs

In [20]:
def Puzzle(heuristic):
    # Initialize OPEN to start state and compute cost
    OPEN = [Node(state=start, parent=None, operator=None)]
    S_goal, _ = SequenceRef(goal) # Reference goal sequence (produce nested list of paired value)
    OPEN[0].cost = ComputeCost(OPEN[0], S_goal, heuristic)

    # Initialize explored node
    CLOSED = []

    # Initialize count for search cost
    nodes_generated = 0

    while True:

        # Failure if empty
        if len(OPEN) == 0:
            return print('Failure')

        # Remove node with smallest cost
        OPEN_states, OPEN_costs = NodeStatesCosts(OPEN)
        min_index = OPEN_costs.index(min(OPEN_costs))
        del OPEN_states[min_index]
        del OPEN_costs[min_index]
        node_n = OPEN.pop(min_index)

        # Explored node
        CLOSED.append(node_n)
        CLOSED_states, CLOSED_costs = NodeStatesCosts(CLOSED)

        # Trace pointers if node is goal
        if node_n.state == goal:
            operators = []
            solution_path = []
            costs = []
            while node_n.parent is not None:
                operators.append(node_n.operator)
                solution_path.append(node_n.state)
                costs.append(node_n.cost)
                node_n = node_n.parent
            operators.reverse()
            solution_path.reverse()
            costs.reverse()
            solution = (operators, solution_path, costs, nodes_generated, node_n.cost)
            return solution # node_n.cost is the cost of start

        # Expand nodes and compute cost for each
        successors = []
        for operator, state in NodeExpansion(node_n.state):
            successor = Node(state=state, parent=node_n, operator=operator)
            successor.cost = ComputeCost(successor, S_goal, heuristic)
            successors.append(successor)
            nodes_generated += 1
        
        # If no succesor, repeat loop
        if len(successors) == 0:
            continue

        # Associate cost
        for successor in successors:
            # Not on either OPEN or CLOSED
            if successor.state not in OPEN_states and successor.state not in CLOSED_states:
                OPEN.append(successor)
            elif successor.state in CLOSED_states:
                CLOSED_index = CLOSED_states.index(successor.state)
                if successor.cost[0] < CLOSED_costs[CLOSED_index][0]:
                    OPEN.append(successor)  # Put on OPEN successors on CLOSED w/ lower cost
                    del CLOSED[CLOSED_index]
            elif successor.state in OPEN_states:
                OPEN_index = OPEN_states.index(successor.state)
                if successor.cost[0] < OPEN_costs[OPEN_index][0]:
                    OPEN[OPEN_index].parent = node_n    # Redirect pointers for lower cost
                    OPEN[OPEN_index].cost = successor.cost 


In [21]:
def DisplayOutput(solution):
    output = ['f(n)', 'g(n)', 'h(n)', 'P(n)', 'S(n)']
    print('start')
    for row in start:
        print(row)
    for i_out in range(len(solution[4])):
        print(output[i_out], ' = ', solution[4][i_out])
    print('\n')
    for move, path, cost in zip(solution[0], solution[1], solution[2]):
        print(move)
        for row in path:
            print(row)
        for j_out in range(len(cost)):
            print(output[j_out], ' = ', cost[j_out])
        print('\n')
    print('Search Cost: ', solution[3])

a) Number of Tiles in the Wrong Position

In [22]:
solution = Puzzle('a')
DisplayOutput(solution)

start
['2', '1', '6']
['4', '*', '8']
['7', '5', '3']
f(n)  =  7
g(n)  =  0
h(n)  =  7


right
['2', '1', '6']
['4', '8', '*']
['7', '5', '3']
f(n)  =  8
g(n)  =  1
h(n)  =  7


up
['2', '1', '*']
['4', '8', '6']
['7', '5', '3']
f(n)  =  9
g(n)  =  2
h(n)  =  7


left
['2', '*', '1']
['4', '8', '6']
['7', '5', '3']
f(n)  =  10
g(n)  =  3
h(n)  =  7


down
['2', '8', '1']
['4', '*', '6']
['7', '5', '3']
f(n)  =  11
g(n)  =  4
h(n)  =  7


right
['2', '8', '1']
['4', '6', '*']
['7', '5', '3']
f(n)  =  12
g(n)  =  5
h(n)  =  7


down
['2', '8', '1']
['4', '6', '3']
['7', '5', '*']
f(n)  =  13
g(n)  =  6
h(n)  =  7


left
['2', '8', '1']
['4', '6', '3']
['7', '*', '5']
f(n)  =  13
g(n)  =  7
h(n)  =  6


up
['2', '8', '1']
['4', '*', '3']
['7', '6', '5']
f(n)  =  13
g(n)  =  8
h(n)  =  5


left
['2', '8', '1']
['*', '4', '3']
['7', '6', '5']
f(n)  =  14
g(n)  =  9
h(n)  =  5


up
['*', '8', '1']
['2', '4', '3']
['7', '6', '5']
f(n)  =  15
g(n)  =  10
h(n)  =  5


right
['8', '*', '1']
['2'

b) Manhattan Distance

In [23]:
solution = Puzzle('b')
DisplayOutput(solution)

start
['2', '1', '6']
['4', '*', '8']
['7', '5', '3']
f(n)  =  12
g(n)  =  0
h(n)  =  12


right
['2', '1', '6']
['4', '8', '*']
['7', '5', '3']
f(n)  =  12
g(n)  =  1
h(n)  =  11


up
['2', '1', '*']
['4', '8', '6']
['7', '5', '3']
f(n)  =  12
g(n)  =  2
h(n)  =  10


left
['2', '*', '1']
['4', '8', '6']
['7', '5', '3']
f(n)  =  14
g(n)  =  3
h(n)  =  11


down
['2', '8', '1']
['4', '*', '6']
['7', '5', '3']
f(n)  =  16
g(n)  =  4
h(n)  =  12


right
['2', '8', '1']
['4', '6', '*']
['7', '5', '3']
f(n)  =  16
g(n)  =  5
h(n)  =  11


down
['2', '8', '1']
['4', '6', '3']
['7', '5', '*']
f(n)  =  16
g(n)  =  6
h(n)  =  10


left
['2', '8', '1']
['4', '6', '3']
['7', '*', '5']
f(n)  =  16
g(n)  =  7
h(n)  =  9


up
['2', '8', '1']
['4', '*', '3']
['7', '6', '5']
f(n)  =  16
g(n)  =  8
h(n)  =  8


left
['2', '8', '1']
['*', '4', '3']
['7', '6', '5']
f(n)  =  16
g(n)  =  9
h(n)  =  7


up
['*', '8', '1']
['2', '4', '3']
['7', '6', '5']
f(n)  =  18
g(n)  =  10
h(n)  =  8


right
['8', '*',

c) Nilsson's Sequence Score

In [24]:
solution = Puzzle('c')
DisplayOutput(solution)

start
['2', '1', '6']
['4', '*', '8']
['7', '5', '3']
f(n)  =  60
g(n)  =  0
h(n)  =  60
P(n)  =  12
S(n)  =  16


right
['2', '1', '6']
['4', '8', '*']
['7', '5', '3']
f(n)  =  57
g(n)  =  1
h(n)  =  56
P(n)  =  11
S(n)  =  15


up
['2', '1', '*']
['4', '8', '6']
['7', '5', '3']
f(n)  =  57
g(n)  =  2
h(n)  =  55
P(n)  =  10
S(n)  =  15


left
['2', '*', '1']
['4', '8', '6']
['7', '5', '3']
f(n)  =  59
g(n)  =  3
h(n)  =  56
P(n)  =  11
S(n)  =  15


down
['2', '8', '1']
['4', '*', '6']
['7', '5', '3']
f(n)  =  58
g(n)  =  4
h(n)  =  54
P(n)  =  12
S(n)  =  14


right
['2', '8', '1']
['4', '6', '*']
['7', '5', '3']
f(n)  =  55
g(n)  =  5
h(n)  =  50
P(n)  =  11
S(n)  =  13


down
['2', '8', '1']
['4', '6', '3']
['7', '5', '*']
f(n)  =  55
g(n)  =  6
h(n)  =  49
P(n)  =  10
S(n)  =  13


left
['2', '8', '1']
['4', '6', '3']
['7', '*', '5']
f(n)  =  55
g(n)  =  7
h(n)  =  48
P(n)  =  9
S(n)  =  13


up
['2', '8', '1']
['4', '*', '3']
['7', '6', '5']
f(n)  =  46
g(n)  =  8
h(n)  =  38
P(